# Initialization

## Import Libraries

In [1]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict
import time

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-12-10 01:23:36.569126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 01:23:36.579413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 01:23:36.580091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Import Custom Module

In [2]:
#adjust filepath so that working directory is folder "recreating-residual-attention-network"
import os

PROJECT_PATH = '/home/ecbm4040/recreating-residual-attention-network'
os.chdir(PROJECT_PATH)

In [ ]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "ECBM4040", "FinalProject", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

In [3]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork, Attention56, Attention92, Attention128, Attention164
from src.utils import generate_data

2021-12-10 01:23:36.700529: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-10 01:23:36.701476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 01:23:36.702215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 01:23:36.702792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

## Custom Callbacks

In [4]:
### Callback to save per epoch time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        # Save epoch time directly on the csv_logger
        logs['per_epoch_time'] = time.time() - self.epoch_time_start

### Callback to stop when certain accuracy is reached
class EarlyStoppingByAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, monitor='accuracy', value=0.98, verbose=0):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

# Modelling CIFAR-10

In [5]:
INPUT_SHAPE, NUM_CLASS, train_ds, val_ds, test_ds, _ = generate_data.get_cifar10()

In [6]:
BATCH_SIZE = 256

In [7]:
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
STEP_SIZE = len(train_ds)

## Naive Attention Learning

In this part, the experiment is done using Naive Attention Learning mechanism

### Attention-56

In [9]:
N_EPOCH = 100
LR = 1e-3

In [10]:
ran_model = Attention56(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple                 

In [11]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/1-a56-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/1-a56-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [12]:
a56_nal_model = ran_model

a56_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a56_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

Epoch 1/100


2021-12-10 01:24:13.362181: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200


157/157 [==============================] - 34s 124ms/step - loss: 2.4147 - accuracy: 0.1653 - val_loss: 2.8198 - val_accuracy: 0.1000 - per_epoch_time: 34.2333
Epoch 2/100
157/157 [==============================] - 17s 107ms/step - loss: 2.1098 - accuracy: 0.2437 - val_loss: 2.4934 - val_accuracy: 0.1345 - per_epoch_time: 16.8126
Epoch 3/100
157/157 [==============================] - 17s 107ms/step - loss: 1.9906 - accuracy: 0.2894 - val_loss: 2.1168 - val_accuracy: 0.2334 - per_epoch_time: 16.8017
Epoch 4/100
157/157 [==============================] - 17s 106ms/step - loss: 1.9104 - accuracy: 0.3109 - val_loss: 1.9089 - val_accuracy: 0.3151 - per_epoch_time: 16.7080
Epoch 5/100
157/157 [==============================] - 17s 107ms/step - loss: 1.8445 - accuracy: 0.3345 - val_loss: 1.8669 - val_accuracy: 0.3407 - per_epoch_time: 16.7772
Epoch 6/100
157/157 [==============================] - 17s 107ms/step - loss: 1.7991 - accuracy: 0.3466 - val_loss: 2.0127 - val_accuracy: 0.3054 - per_

157/157 [==============================] - 17s 105ms/step - loss: 0.9869 - accuracy: 0.6624 - val_loss: 1.2141 - val_accuracy: 0.6094 - per_epoch_time: 16.5273
Epoch 49/100
157/157 [==============================] - 17s 106ms/step - loss: 0.9740 - accuracy: 0.6666 - val_loss: 1.1655 - val_accuracy: 0.6149 - per_epoch_time: 16.7191
Epoch 50/100
157/157 [==============================] - 17s 106ms/step - loss: 0.9706 - accuracy: 0.6690 - val_loss: 1.1819 - val_accuracy: 0.6045 - per_epoch_time: 16.6194
Epoch 51/100
157/157 [==============================] - 17s 107ms/step - loss: 0.9561 - accuracy: 0.6761 - val_loss: 1.0845 - val_accuracy: 0.6450 - per_epoch_time: 16.7798
Epoch 52/100
157/157 [==============================] - 16s 105ms/step - loss: 0.9405 - accuracy: 0.6794 - val_loss: 1.0293 - val_accuracy: 0.6553 - per_epoch_time: 16.4355
Epoch 53/100
157/157 [==============================] - 17s 106ms/step - loss: 0.9253 - accuracy: 0.6852 - val_loss: 1.1156 - val_accuracy: 0.6389 -

157/157 [==============================] - 17s 107ms/step - loss: 0.4888 - accuracy: 0.8375 - val_loss: 0.6446 - val_accuracy: 0.7978 - per_epoch_time: 16.7214
Epoch 96/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4873 - accuracy: 0.8392 - val_loss: 0.6624 - val_accuracy: 0.7909 - per_epoch_time: 16.5859
Epoch 97/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4773 - accuracy: 0.8403 - val_loss: 0.6616 - val_accuracy: 0.7918 - per_epoch_time: 16.6485
Epoch 98/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4821 - accuracy: 0.8407 - val_loss: 0.6460 - val_accuracy: 0.7960 - per_epoch_time: 16.8286
Epoch 99/100
157/157 [==============================] - 17s 108ms/step - loss: 0.4777 - accuracy: 0.8423 - val_loss: 0.6440 - val_accuracy: 0.7961 - per_epoch_time: 16.9069
Epoch 100/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4779 - accuracy: 0.8410 - val_loss: 0.6464 - val_accuracy: 0.7958 

### Attention-92

Decrease initial learning rate and add gradient clipping in optimizer to avoid exploding gradient

In [43]:
N_EPOCH = 100
LR = 1e-4

In [44]:
ran_model = Attention92(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention92_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_2049 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_1572 (B  multiple                 128       
 atchNormalization)                                              
                                                                 
 re_lu_1572 (ReLU)           multiple                  0         
                                                                 
 max_pooling2d_105 (MaxPooli  multiple                 0         
 ng2D)                                                           
                                                                 
 residual_unit_486 (Residual  multiple               

In [45]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/2-a92-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/2-a92-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.1)

In [46]:
a92_nal_model = ran_model

a92_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a92_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 59s 211ms/step - loss: 2.4605 - accuracy: 0.1050 - val_loss: 2.3032 - val_accuracy: 0.1000 - per_epoch_time: 58.8410
Epoch 2/100
157/157 [==============================] - 30s 189ms/step - loss: 2.3990 - accuracy: 0.1122 - val_loss: 2.3080 - val_accuracy: 0.1000 - per_epoch_time: 29.6451
Epoch 3/100
157/157 [==============================] - 30s 190ms/step - loss: 2.3808 - accuracy: 0.1239 - val_loss: 2.3437 - val_accuracy: 0.1000 - per_epoch_time: 29.7942
Epoch 4/100
157/157 [==============================] - 30s 189ms/step - loss: 2.3604 - accuracy: 0.1306 - val_loss: 2.3259 - val_accuracy: 0.1377 - per_epoch_time: 29.6975
Epoch 5/100
157/157 [==============================] - 29s 188ms/step - loss: 2.3501 - accuracy: 0.1348 - val_loss: 2.3805 - val_accuracy: 0.1498 - per_epoch_time: 29.4914
Epoch 6/100
157/157 [==============================] - 29s 187ms/step - loss: 2.3432 - accuracy: 0.1377 - val_loss: 2.6593 - val_accuracy: 0

157/157 [==============================] - 29s 187ms/step - loss: 1.6943 - accuracy: 0.3893 - val_loss: 1.6312 - val_accuracy: 0.4099 - per_epoch_time: 29.3529
Epoch 49/100
157/157 [==============================] - 29s 187ms/step - loss: 1.6803 - accuracy: 0.3953 - val_loss: 1.6881 - val_accuracy: 0.3858 - per_epoch_time: 29.4238
Epoch 50/100
157/157 [==============================] - 29s 188ms/step - loss: 1.6610 - accuracy: 0.3999 - val_loss: 1.6751 - val_accuracy: 0.4002 - per_epoch_time: 29.4321
Epoch 51/100
157/157 [==============================] - 30s 191ms/step - loss: 1.6529 - accuracy: 0.4080 - val_loss: 1.6188 - val_accuracy: 0.4213 - per_epoch_time: 29.9042
Epoch 52/100
157/157 [==============================] - 30s 190ms/step - loss: 1.6369 - accuracy: 0.4108 - val_loss: 1.6203 - val_accuracy: 0.4180 - per_epoch_time: 29.8630
Epoch 53/100
157/157 [==============================] - 29s 187ms/step - loss: 1.6361 - accuracy: 0.4117 - val_loss: 1.5993 - val_accuracy: 0.4382 -

157/157 [==============================] - 29s 188ms/step - loss: 1.2906 - accuracy: 0.5360 - val_loss: 1.4025 - val_accuracy: 0.5241 - per_epoch_time: 29.4451
Epoch 96/100
157/157 [==============================] - 30s 189ms/step - loss: 1.2910 - accuracy: 0.5390 - val_loss: 1.4640 - val_accuracy: 0.5269 - per_epoch_time: 29.7068
Epoch 97/100
157/157 [==============================] - 30s 190ms/step - loss: 1.2871 - accuracy: 0.5375 - val_loss: 1.4564 - val_accuracy: 0.5225 - per_epoch_time: 29.8167
Epoch 98/100
157/157 [==============================] - 30s 190ms/step - loss: 1.2843 - accuracy: 0.5416 - val_loss: 1.4070 - val_accuracy: 0.5235 - per_epoch_time: 29.7661
Epoch 99/100
157/157 [==============================] - 30s 191ms/step - loss: 1.2861 - accuracy: 0.5397 - val_loss: 1.4047 - val_accuracy: 0.5279 - per_epoch_time: 29.9327
Epoch 100/100
157/157 [==============================] - 30s 190ms/step - loss: 1.2823 - accuracy: 0.5391 - val_loss: 1.4251 - val_accuracy: 0.5260 

### Attention-128

In [ ]:
N_EPOCH = 100
LR = 1e-3

In [ ]:
ran_model = Attention128(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/3-a128-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/3-a128-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
a128_nal_model = ran_model

a128_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a128_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

### Attention-164

In [ ]:
N_EPOCH = 100
LR = 1e-3

In [ ]:
ran_model = Attention164(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/4-a192-nal-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/4-a192-nal-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
a192_nal_model = ran_model

a192_nal_model.compile(optimizer=our_optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(), 
                      metrics=['accuracy'])


history = a192_nal_model.fit(train_ds,
                            validation_data=val_ds,
                            callbacks=callbacks,
                            epochs=N_EPOCH, verbose=1)

## Attention-Residual Learning

In this part, the experiment is done using Residual Attention Learning mechanism

### Attention-56

In [13]:
N_EPOCH = 100
LR = 1e-3

In [14]:
ran_model = Attention56(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention56_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple               

In [15]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/1-a56-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/1-a56-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [16]:
a56_model = ran_model

a56_model.compile(optimizer=our_optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a56_model.fit(train_ds,
                        validation_data=val_ds,
                        callbacks=callbacks,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 31s 119ms/step - loss: 2.4594 - accuracy: 0.1515 - val_loss: 2.6538 - val_accuracy: 0.1000 - per_epoch_time: 30.6641
Epoch 2/100
157/157 [==============================] - 17s 107ms/step - loss: 2.0942 - accuracy: 0.2455 - val_loss: 2.5194 - val_accuracy: 0.1320 - per_epoch_time: 16.7516
Epoch 3/100
157/157 [==============================] - 17s 108ms/step - loss: 1.9156 - accuracy: 0.3052 - val_loss: 1.9570 - val_accuracy: 0.2832 - per_epoch_time: 16.8760
Epoch 4/100
157/157 [==============================] - 17s 106ms/step - loss: 1.8190 - accuracy: 0.3408 - val_loss: 1.7478 - val_accuracy: 0.3669 - per_epoch_time: 16.6506
Epoch 5/100
157/157 [==============================] - 17s 107ms/step - loss: 1.7587 - accuracy: 0.3671 - val_loss: 1.9506 - val_accuracy: 0.3231 - per_epoch_time: 16.8248
Epoch 6/100
157/157 [==============================] - 17s 109ms/step - loss: 1.7030 - accuracy: 0.3837 - val_loss: 1.8401 - val_accuracy: 0

157/157 [==============================] - 17s 108ms/step - loss: 0.8919 - accuracy: 0.7015 - val_loss: 1.0953 - val_accuracy: 0.6427 - per_epoch_time: 16.9039
Epoch 49/100
157/157 [==============================] - 17s 108ms/step - loss: 0.8715 - accuracy: 0.7072 - val_loss: 1.1462 - val_accuracy: 0.6373 - per_epoch_time: 16.8665
Epoch 50/100
157/157 [==============================] - 17s 107ms/step - loss: 0.8672 - accuracy: 0.7092 - val_loss: 1.0143 - val_accuracy: 0.6731 - per_epoch_time: 16.7974
Epoch 51/100
157/157 [==============================] - 17s 108ms/step - loss: 0.8449 - accuracy: 0.7183 - val_loss: 0.9151 - val_accuracy: 0.6965 - per_epoch_time: 16.9208
Epoch 52/100
157/157 [==============================] - 17s 107ms/step - loss: 0.8498 - accuracy: 0.7167 - val_loss: 0.9643 - val_accuracy: 0.6894 - per_epoch_time: 16.7950
Epoch 53/100
157/157 [==============================] - 17s 108ms/step - loss: 0.8303 - accuracy: 0.7221 - val_loss: 1.0478 - val_accuracy: 0.6729 -

157/157 [==============================] - 17s 106ms/step - loss: 0.4640 - accuracy: 0.8471 - val_loss: 0.6167 - val_accuracy: 0.8008 - per_epoch_time: 16.6375
Epoch 96/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4567 - accuracy: 0.8495 - val_loss: 0.6008 - val_accuracy: 0.8041 - per_epoch_time: 16.8036
Epoch 97/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4642 - accuracy: 0.8476 - val_loss: 0.5895 - val_accuracy: 0.8070 - per_epoch_time: 16.7101
Epoch 98/100
157/157 [==============================] - 17s 107ms/step - loss: 0.4549 - accuracy: 0.8497 - val_loss: 0.6205 - val_accuracy: 0.8021 - per_epoch_time: 16.7496
Epoch 99/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4526 - accuracy: 0.8509 - val_loss: 0.6155 - val_accuracy: 0.8012 - per_epoch_time: 16.6782
Epoch 100/100
157/157 [==============================] - 17s 106ms/step - loss: 0.4524 - accuracy: 0.8512 - val_loss: 0.6105 - val_accuracy: 0.8047 

### Attention-92

Decrease initial learning rate and add gradient clipping in optimizer to avoid exploding gradient

In [47]:
N_EPOCH = 100
LR = 1e-4

In [48]:
ran_model = Attention92(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention92_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_2302 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_1766 (B  multiple                 128       
 atchNormalization)                                              
                                                                 
 re_lu_1766 (ReLU)           multiple                  0         
                                                                 
 max_pooling2d_118 (MaxPooli  multiple                 0         
 ng2D)                                                           
                                                                 
 residual_unit_546 (Residual  multiple               

In [49]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/2-a92-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/2-a92-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=0.1)

In [50]:
a92_model = ran_model

a92_model.compile(optimizer=our_optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a92_model.fit(train_ds,
                        validation_data=val_ds,
                        callbacks=callbacks,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/100
157/157 [==============================] - 60s 213ms/step - loss: 2.4732 - accuracy: 0.1113 - val_loss: 2.3049 - val_accuracy: 0.1000 - per_epoch_time: 59.5054
Epoch 2/100
157/157 [==============================] - 30s 190ms/step - loss: 2.4109 - accuracy: 0.1159 - val_loss: 2.3279 - val_accuracy: 0.1000 - per_epoch_time: 29.7945
Epoch 3/100
157/157 [==============================] - 30s 191ms/step - loss: 2.3843 - accuracy: 0.1195 - val_loss: 2.3510 - val_accuracy: 0.0998 - per_epoch_time: 29.9325
Epoch 4/100
157/157 [==============================] - 30s 192ms/step - loss: 2.3722 - accuracy: 0.1249 - val_loss: 2.2706 - val_accuracy: 0.1462 - per_epoch_time: 30.1086
Epoch 5/100
157/157 [==============================] - 30s 191ms/step - loss: 2.3585 - accuracy: 0.1307 - val_loss: 2.3331 - val_accuracy: 0.1455 - per_epoch_time: 29.9106
Epoch 6/100
157/157 [==============================] - 30s 190ms/step - loss: 2.3412 - accuracy: 0.1328 - val_loss: 2.3906 - val_accuracy: 0

157/157 [==============================] - 30s 190ms/step - loss: 1.5757 - accuracy: 0.4392 - val_loss: 1.5397 - val_accuracy: 0.4493 - per_epoch_time: 29.8221
Epoch 49/100
157/157 [==============================] - 30s 189ms/step - loss: 1.5555 - accuracy: 0.4443 - val_loss: 1.6116 - val_accuracy: 0.4302 - per_epoch_time: 29.7062
Epoch 50/100
157/157 [==============================] - 30s 190ms/step - loss: 1.5615 - accuracy: 0.4450 - val_loss: 1.5567 - val_accuracy: 0.4455 - per_epoch_time: 29.8268
Epoch 51/100
157/157 [==============================] - 30s 189ms/step - loss: 1.5467 - accuracy: 0.4475 - val_loss: 1.5766 - val_accuracy: 0.4418 - per_epoch_time: 29.6843
Epoch 52/100
157/157 [==============================] - 30s 189ms/step - loss: 1.5273 - accuracy: 0.4548 - val_loss: 1.5872 - val_accuracy: 0.4499 - per_epoch_time: 29.6446
Epoch 53/100
157/157 [==============================] - 30s 189ms/step - loss: 1.5196 - accuracy: 0.4599 - val_loss: 1.5502 - val_accuracy: 0.4413 -

157/157 [==============================] - 30s 190ms/step - loss: 1.1675 - accuracy: 0.5790 - val_loss: 1.3552 - val_accuracy: 0.5549 - per_epoch_time: 29.7521
Epoch 96/100
157/157 [==============================] - 30s 189ms/step - loss: 1.1677 - accuracy: 0.5822 - val_loss: 1.2542 - val_accuracy: 0.5628 - per_epoch_time: 29.6772
Epoch 97/100
157/157 [==============================] - 30s 189ms/step - loss: 1.1677 - accuracy: 0.5832 - val_loss: 1.2983 - val_accuracy: 0.5572 - per_epoch_time: 29.6808
Epoch 98/100
157/157 [==============================] - 30s 189ms/step - loss: 1.1707 - accuracy: 0.5820 - val_loss: 1.2802 - val_accuracy: 0.5632 - per_epoch_time: 29.6716
Epoch 99/100
157/157 [==============================] - 29s 186ms/step - loss: 1.1562 - accuracy: 0.5834 - val_loss: 1.3748 - val_accuracy: 0.5450 - per_epoch_time: 29.1147
Epoch 100/100
157/157 [==============================] - 29s 187ms/step - loss: 1.1566 - accuracy: 0.5846 - val_loss: 1.3151 - val_accuracy: 0.5638 

### Attention-128

In [13]:
ran_model = Attention128(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_392 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_301 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_301 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_20 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_93 (ResidualU  multiple                

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/3-a128-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/3-a128-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [16]:
a128_model = ran_model

a128_model.compile(optimizer=our_optimizer, 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a128_model.fit(train_ds,
                         validation_data=val_ds,
                         callbacks=callbacks,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 97s 425ms/step - loss: 2.3997 - accuracy: 0.1120 - val_loss: 2.3032 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 62s 394ms/step - loss: 2.3201 - accuracy: 0.1385 - val_loss: 2.3081 - val_accuracy: 0.1000
Epoch 3/10
157/157 [==============================] - 62s 394ms/step - loss: 2.3025 - accuracy: 0.1425 - val_loss: 2.3207 - val_accuracy: 0.1014
Epoch 4/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2850 - accuracy: 0.1495 - val_loss: 2.2386 - val_accuracy: 0.1558
Epoch 5/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2778 - accuracy: 0.1456 - val_loss: 2.2371 - val_accuracy: 0.1636
Epoch 6/10
157/157 [==============================] - 62s 394ms/step - loss: 2.2649 - accuracy: 0.1521 - val_loss: 2.3654 - val_accuracy: 0.1311
Epoch 7/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2690 - accuracy: 0.1529 - val_loss: 2.3923 - val_ac

### Attention-164

In [14]:
ran_model = Attention164(input_shape=INPUT_SHAPE, num_class=NUM_CLASS)
inputs = tf.keras.Input(INPUT_SHAPE)
ran_model(inputs)
ran_model.summary()

Model: "attention164"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_759 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_582 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_582 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_39 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_180 (Residual  multiple                

In [ ]:
## Callbacks
time_callback = TimeHistory()

# checkpointing & logger
model_checkpointer = ModelCheckpoint(filepath="data/saved_weights/4-a192-cifar10.h5",
                                     verbose=0,
                                     save_weights_only=True,
                                     save_best_only=False)

csv_logger = CSVLogger('data/logs/4-a192-cifar10.csv')

callbacks = [model_checkpointer, time_callback, csv_logger]

## Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[80*STEP_SIZE, 120*STEP_SIZE, 160*STEP_SIZE, 180*STEP_SIZE],
    values=[LR, 0.1*LR, 1e-2*LR, 1e-3*LR, 0.5e-3*LR]
)

our_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [17]:
a164_model = ran_model

a164_model.compile(optimizer=our_optimizer, 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a164_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 98s 425ms/step - loss: 2.1079 - accuracy: 0.1941 - val_loss: 2.1368 - val_accuracy: 0.1748
Epoch 2/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9617 - accuracy: 0.2410 - val_loss: 2.9659 - val_accuracy: 0.1587
Epoch 3/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9016 - accuracy: 0.2630 - val_loss: 3.4470 - val_accuracy: 0.1594


KeyboardInterrupt: ignored